In [2]:
import json
import torch
from datasets.custom_dataset import CustomDataset
from datasets.transform import TransformSelector
from models.model_selector import ModelSelector
from utils.train_utils import Trainer
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm
import cv2

import matplotlib
import matplotlib.pyplot as plt

In [3]:
# 추론(inference) 함수
def inference(model, device, test_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images in tqdm(test_loader, desc="Inference"):
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            predictions.extend(preds.cpu().numpy())
    return predictions

# 'best_model' 파일을 찾는 함수 (가장 최근에 저장된 파일 선택)
def get_best_model_path(directory):
    files = [f for f in os.listdir(directory) if f.startswith('best_model') and f.endswith('.pt')]
    if not files:
        raise FileNotFoundError(f"No best model files found in directory: {directory}")
    
    # 파일의 수정 시간을 기준으로 가장 최근에 저장된 파일 선택
    best_file = max(files, key=lambda f: os.path.getmtime(os.path.join(directory, f)))
    return os.path.join(directory, best_file)

def validate(model, device, val_loader):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.08)
    total_batches = len(val_loader)
    
    progress_bar = tqdm(val_loader, desc="Validating", leave=False)
    
    with torch.no_grad():
        for batch_idx, (images, targets, indices) in enumerate(progress_bar):
            images, targets = images.to(device), targets.to(device)
            
            # 모델 예측
            outputs = model(images)
            outputs_softmax = torch.nn.functional.softmax(outputs)
            
            # 손실 계산
            loss = loss_fn(outputs, targets)
            total_loss += loss.item()
            
            # 정확도 계산
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
            
            progress_bar.set_postfix({'val_batch_loss': loss.item()})

            # 이미지 그리기
            top_fives, top_fives_indices = torch.topk(outputs_softmax, 5)

            images = images.cpu().numpy()
            targets = targets.cpu().numpy()
            indices = indices.cpu().numpy()
            outputs_softmax = outputs_softmax.cpu().numpy()
            top_fives = top_fives.cpu().numpy()
            top_fives_indices = top_fives_indices.cpu().numpy()                    
    
    # 검증의 평균 손실과 정확도 계산
    avg_loss = total_loss / total_batches
    accuracy = correct / total
    
    print(f"Validation Epoch Average Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
    
    return avg_loss, accuracy

def convert_image_for_display(image):
    # neural network를 위한 이미지에서 plt.imshow()를 위한 이미지 변환
    # plt.imshow()는 float 데이터에 대해 [0, 1] 구간을 truncate해서 display 하기 때문에 normalization이 필요함
    image = image - image.min()
    image = image / image.max()
    image = image.transpose(1, 2, 0)
    
    return image

In [4]:
setup = {
  "coatnet": r"/data/ephemeral/home/Dongjin/git/level1-imageclassification-cv-07/LV1/config_dj1.json",
  "ViT": r"/data/ephemeral/home/Dongjin/git/level1-imageclassification-cv-07/LV1/config_dj6.json",
  "resnet": r"/data/ephemeral/home/Dongjin/git/level1-imageclassification-cv-07/LV1/config_dj8.json",
  }

config_path = setup["resnet"]
with open(config_path, 'r') as f:
  config = json.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_info = pd.read_csv(config['data_info_file'])

py_dir_path = r"/data/ephemeral/home/Dongjin/git/level1-imageclassification-cv-07/Model/0921_valid_analysis"
rel_train_index_path = r"datasets/train_index.csv"
rel_val_index_path = r"datasets/val_index.csv"

train_index_path = os.path.join(py_dir_path, rel_train_index_path)
val_index_path = os.path.join(py_dir_path, rel_val_index_path)

# train_index.csv와 val_index.csv를 이용하여 train_df와 val_df를 로드       
train_index = pd.read_csv(train_index_path, header = None).squeeze()
val_index = pd.read_csv(val_index_path, header = None).squeeze()

train_df = train_info.loc[train_index]
val_df = train_info.loc[val_index]

transform_selector = TransformSelector(transform_type="albumentations")
val_transform = transform_selector.get_transform(is_train=False)

val_dataset = CustomDataset(root_dir=config['train_data_dir'], info_df=val_df, transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

# 모델 설정
model_selector = ModelSelector(model_type="timm", num_classes=config['num_classes'], model_name=config['model_name'], pretrained=False)
model = model_selector.get_model()

# 베스트 모델 경로 설정
model_path = get_best_model_path(config['result_path'])
print(f"Loading best model from {model_path}")

# 저장된 모델 로드
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)

Loading best model from /data/ephemeral/home/Dongjin/git/level1-imageclassification-cv-07/LV1/results/resnet101/best_model_1.6079.pt


TimmModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act2): ReLU(inplace=True)
        (aa): Identity()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, mome

In [14]:
matplotlib.use('Agg')

def plot(model_name, images, targets, dataset_indices,outputs_softmax, top_fives, top_fives_indices, predicts):
    correct_save_fold = f'result/{model_name}/correct'
    wrong_save_fold = f'result/{model_name}/wrong'

    for i in range(images.shape[0]):
        image = images[i]
        target = targets[i]
        dataset_index = dataset_indices[i] 
        output_softmax = outputs_softmax[i]
        top_five = top_fives[i]
        top_fives_index = top_fives_indices[i]
        predict = predicts[i]
        iscorrect = target == predict
        
        img_rel_path = train_info['image_path'].loc[dataset_index]
        img_path = os.path.join(config['train_data_dir'], img_rel_path)
        class_name = train_info['class_name'].loc[dataset_index]

        fig, axes = plt.subplots(1, 2, figsize = (10, 6))
        image_origin = cv2.imread(img_path, cv2.IMREAD_COLOR)
        image_origin = cv2.cvtColor(image_origin, cv2.COLOR_BGR2RGB)
        image_input = convert_image_for_display(image) # network input image

        axes[0].imshow(image_origin)
        axes[1].imshow(image_input)

        axes[0].set_title("Origin")
        axes[1].set_title("Common transformation")

        for i in range(2):
            axes[i].axis('off')

        # title 만들기
        if iscorrect == True:
            title0 = f"{model_name} - correct prediction - target: {target}, predict: {predict}"
        else:
            title0 = f"{model_name} - wrong prediction - target: {target}, predict: {predict}"

        title1 = ""
        for index, prob in zip(top_fives_index, top_five * 100):
            if (title1 == ""):
                title1 = f"Top 5 - {index}:{prob:.1f}%"
            else:
                title1 = title1 + ", " + f"{index}:{prob:.1f}%"

        title = f"{title0}\n{title1}\n{img_rel_path}"
        plt.suptitle(title)

        # 저장하기

        if iscorrect:
            save_path = os.path.join(correct_save_fold, f"{target}/{dataset_index}.png")
        else:
            save_path = os.path.join(wrong_save_fold, f"{target}/{dataset_index}.png")

        if not os.path.isdir(os.path.split(save_path)[0]):
            os.makedirs(os.path.split(save_path)[0])

        plt.savefig(save_path)

In [15]:
model.eval()
total_loss = 0.0
correct = 0
total = 0

loss_fn = nn.CrossEntropyLoss(label_smoothing=0.08)
total_batches = len(val_loader)

progress_bar = tqdm(val_loader, desc="Validating", leave=False)

with torch.no_grad():
    for batch_idx, (images, targets, dataset_indices) in enumerate(progress_bar):
        images, targets = images.to(device), targets.to(device)
        
        # 모델 예측
        outputs = model(images)
        outputs_softmax = torch.nn.functional.softmax(outputs)
        
        # 손실 계산
        loss = loss_fn(outputs, targets)
        total_loss += loss.item()
        
        # 정확도 계산
        _, predicts = torch.max(outputs, 1)
        correct += (predicts == targets).sum().item()
        total += targets.size(0)
        
        progress_bar.set_postfix({'val_batch_loss': loss.item()})

        # 이미지 그리기
        top_fives, top_fives_indices = torch.topk(outputs_softmax, 5)

        images = images.cpu().numpy()
        targets = targets.cpu().numpy()
        dataset_indices = dataset_indices.cpu().numpy()
        outputs_softmax = outputs_softmax.cpu().numpy()
        top_fives = top_fives.cpu().numpy()
        top_fives_indices = top_fives_indices.cpu().numpy()
        predicts = predicts.cpu().numpy()

        plot(config['model_name'], images, targets, dataset_indices,outputs_softmax, top_fives, top_fives_indices, predicts)

Validating:   0%|          | 0/94 [00:00<?, ?it/s]/tmp/ipykernel_519594/3571579520.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs_softmax = torch.nn.functional.softmax(outputs)


In [5]:
rel_val_index_path

'datasets/val_index.csv'